# Motivation

- *What is your dataset?*

Our dataset is the **[MTA Subway Hourly Ridership](https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-2020-2024/wujg-7c2s/about_data)** dataset provided by the Metropolitan Transportation Authority (MTA) of New York. It includes **hourly counts of entries and exits for every subway station in NYC** from **January 2020 to the present**. Each row represents one station-hour, with metadata including date, station name, line, borough, and ridership numbers. The dataset captures both weekday and weekend traffic, making it useful for analysing temporal patterns in New York’s subway system over time.

- *Why did you choose this/these particular dataset(s)?*

We chose this dataset for a few key reasons:

1. **Urban Relevance:** NYC’s subway is the beating heart of the city. Understanding how ridership has changed over time gives insights into broader social, economic, and public health trends—especially during and after the COVID-19 pandemic.

2. **Temporal Depth:** The dataset covers a unique historical period—early 2020 to 2024—which includes a global pandemic, lockdowns, phased reopening, and recovery. This provides a *rare* opportunity to observe real-time responses to major societal events.

3. **Granularity:** The hourly format makes it perfect for exploring:
   - Rush hour patterns 🕓
   - Weekend vs. weekday trends 🗓️
   - Pandemic dips and rebounds 📉📈
   - Differences across neighbourhoods or boroughs 🌆

4. **Storytelling Power:** With so much data at a granular level, we knew we could surface compelling visualizations and micro-stories—whether about how commuters disappeared overnight in March 2020 or how certain stations (e.g., Times Square vs. neighbourhood stops) bounced back at different speeds.

- *What was your goal for the end user's experience?*

Our goal was to **make the data human**.

We wanted our audience—whether they're New Yorkers, urban planners, or just data-curious readers—to *feel* the story behind the numbers. So we focused on:

- **Clear, interactive visuals** that let users explore ridership over time, zoom into specific stations, and see how different neighbourhoods were affected.
- **Contextual storytelling** that pairs visuals with explanations—highlighting key events like the March 2020 lockdown, vaccine rollout periods, and seasonal shifts.
- **Empathy + Insight:** We wanted users to think:
  _“Oh wow, look how empty Grand Central got in April 2020…”_
  or  
  _“Interesting, weekend ridership recovered faster than weekday commuting!”_

Ultimately, we aimed to create a *narrative journey*—not just a dashboard—so users could explore the rhythm of NYC’s subways and how it reflects the resilience and evolution of the city itself.


# Basic Stats

- *Write about your choices in data cleaning and preprocessing*.


#### **Data Cleaning**

**Which columns can we remove?**


**Which rows of columns should we remove?**

- *Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.*


# Data Analysis

- *Describe your data analysis and explain what you've learned about the dataset.*
- *If relevant, talk about your machine-learning.*

##### **FOR CLARITY OF THE EXPLAINER, THE DATA ANALYSIS IS DONE IN SEPARATE NOTEBOOKS AND REFERENCED. ONLY SELECT PLOTS ARE SHOWN HERE**

The data analysis plots are done in different notebooks as they are too large to have in one notebook. Selected plots are shown in this notebook to give an idea of the most important findings.
We know that you asked for the explainer notebook to contain all the code that made up this project but we truly believe that using our current folder structure with hyperlinks and reference links to the respective files are much manageable and clear to individuals grading this project

### Temporal Data Analysis

The full temporal data analysis is done in file [Temporal Data Analysis Code](data_analysis_and_preprocessing/temporal_plots.ipynb). Firstly, an analysis of all temporal changes for all top 25 most common crimes is done by Year, Month, Weekday, 24-Hour using multipanel bar plots. Here is an example of the Weekday and 24 plots as these were most relevant for travelling in NYC, since the yearly plot is hard to act upon, whereas a more managable hourly and weekday plot can be used to avoid crime hotspots. 

### Bar Plots

### Line And Polar Plots

These plots illustrate that there are significant temporal changes across all 

## Socioeconomic Data Analysis

# Genre

Which genre of data story did you use?

We used the **"Magazine Style" genre** of data storytelling.  

This genre is characterized by a **single-scrollable page** that combines visualizations with text narrative in a structured layout. It’s designed to be **reader-friendly and engaging** for a general audience, much like a longform web article from *The New York Times* or *The Pudding*.  

We chose this genre because it balances **exploration and explanation**—it allows us to guide the reader through a curated journey while offering moments of interactivity and discovery.

- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?

#### 1. **Visual Structure**
- ✅ **Structured Path**: We organized the page linearly with sections like Introduction → Time-Based Trends → Geographic Patterns → Station-Level Deep Dives → Conclusion.  
  *Why?* It helps the reader stay oriented and absorb information one chunk at a time, mimicking how a magazine story unfolds.

- ✅ **Consistent Visual Design**: All visualizations used a consistent color palette and typography.  
  *Why?* To give the piece a polished, cohesive feel and to reduce cognitive load.

#### 2. **Interactivity**
- ✅ **Filtering & Zooming (in Bokeh)**: Users could filter by station or time period, and zoom in on interesting date ranges.  
  *Why?* This gives users the freedom to investigate areas of personal relevance—especially useful in a city like NYC where local context matters.

- ✅ **Hover Highlights / Tooltips**: On maps and line charts, hovering reveals precise data points.  
  *Why?* It keeps the interface clean while providing extra detail on demand.

#### 3. **Annotations**
- ✅ **Textual Callouts**: We used inline callouts to highlight major ridership drops (e.g., March 2020) or interesting rebounds.  
  *Why?* Annotations provide important context that raw numbers alone can't convey—especially during historic periods like the pandemic.

- ✅ **Labels and Icons**: On key charts, we labelled spikes or valleys with short explanations.  
  *Why?* These guide the eye and anchor the story in specific events.

- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

#### 1. **Ordering**
- ✅ **Linear Narrative**: The story flows top-to-bottom with a fixed order.  
  *Why?* This mirrors how people naturally read and is perfect for telling a story with a beginning (normal life), middle (COVID disruption), and end (recovery).

#### 2. **Highlighting**
- ✅ **Textual Emphasis**: We emphasized key dates (like lockdowns or holidays) in the text to guide attention.  
- ✅ **Graphical Highlights**: Spikes/dips in ridership were marked in bold or with different colors on charts.  
  *Why?* These draw the reader’s focus to the "beats" of the story without overwhelming them with detail.

#### 3. **Messaging**
- ✅ **Integrated Explanations**: Each figure had a caption and surrounding text that interpreted the data for the reader.  
  *Why?* We didn’t want readers guessing what a dip or spike meant—they're told directly, so interpretation is easier.

- ✅ **Progressive Disclosure**: We gradually revealed more complex information—starting broad (overall ridership trends) before drilling down into stations and hours.
  *Why?* This scaffolds understanding and prevents overwhelm.

### ✅ Final Note  
By using the **Magazine Style genre** and thoughtfully applying tools across **Visual Narrative** and **Narrative Structure**, we crafted a story that is informative, visually engaging, and emotionally resonant—especially during a time when public transportation was at the center of massive societal shifts.

## Visual Narrative

#### **Visual Structuring**

#### **Highlighting**


#### **Transition Guidance**



## Narrative Structure

#### **Ordering**


#### **Interactivity**


#### **Messaging**





# Visualizations
- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?

##### **Bar Charts**


##### **Line & Polar Plot**


##### **Bokeh Plots**


# Discussion
- What went well?,
- What is still missing? What could be improved?, Why?

### What went well?,
- t
- h

### What is still missing? What could be improved?, Why?
- o
- i


# Contributions

You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
It is not OK simply to write "All group members contributed equally".

##### **General Notes on collaboration**

All 3 students have contributed equally to this project and we have assisted each other and understand each component of the final result. 

However, as required by DTU the following is an outline of the primary responsibilities for each student: